<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/ScraptruffleshuffleLoadMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.P4R248M31M/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.ozWLNvLr3x/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.bEpCMtTZLW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
import pandas as pd
import multiprocessing as mp
from selenium.webdriver.support.ui import WebDriverWait
import time

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
url = "https://www.truffleshuffle.co.uk/"
content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')
navigation = soup.find('ul', class_='layoutInner')
link_tag = navigation.find_all('a')
all_category_link = []
for link in link_tag:
  base_link = "https://www.truffleshuffle.co.uk"
  full_link = base_link + link['href']
  all_category_link.append(full_link)
len(all_category_link)

37

In [41]:
import json
def post_character(image, title_head,  final_description):
  try:
    description = title_head + " " + final_description
    url = 'https://jormungandr.wizewerx.tech/eat'
    headers = {
        'Authorization': 'p7RF3TwoFtaQ9ymoFMS3ZdYfSdc7hEoYedXxZ1L5l7vSXf59ntjvZTymoF5rfpy8RSpD4IErZt2ykSv64Xbu86l7',
        'Content-Type': 'application/json'
    }

    data = {
        "topic": "fuk",
        "description": description,
        "payload": {},
        "files": image
    }
    json_data = json.dumps(data)
    response = requests.post(url, headers=headers, data=json_data)
    if response.status_code == 200:
        pass
    else:
        pass
  except:
    pass

In [36]:
def get_link_details(url):
    content_main = requests.get(url).content
    page = BeautifulSoup(content_main, 'html.parser')
    images_container = page.find('ul', id="thumbnailsContainer")
    images_pictures = images_container.find_all('picture')
    final_images = []
    for image in images_pictures:
      image1 = image.find('source')
      final_images.append(image1['data-defer-src'])
    title = page.find('h1')
    title = title.text
    description = page.find('p', class_="topDescription")
    description = description.text.strip()
    post_character( final_images, title,  description)
    # return final_images, title,  description

In [5]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException

def scrape_truffleshuffle(url):
    driver.get(url)

    try:
        # Check if the "See More" button exists
        see_more_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "loadMore"))
        )

        # Scroll to the "See More" button
        driver.execute_script("arguments[0].scrollIntoView();", see_more_button)

        # Click the "See More" button until it is no longer visible
        while see_more_button.is_displayed():
            try:
                see_more_button.click()
                time.sleep(2)
            except ElementClickInterceptedException:
                # Handle click interception, e.g., scroll again or wait for an overlay to disappear
                pass

        # Find all product cards
        card_product = driver.find_elements(By.CLASS_NAME, "isotopeElement.isotopeElementDefault.productListingBox")

        # Extract the product URLs
        product_urls = []
        for element in card_product:
            product_urls.append(element.get_attribute("href"))

        return product_urls
    except (NoSuchElementException, TimeoutException):
        # If the "See More" button doesn't exist or times out, collect product URLs on the current page
        card_product = driver.find_elements(By.CLASS_NAME, "isotopeElement.isotopeElementDefault.productListingBox")

        product_urls = []
        for element in card_product:
            product_urls.append(element.get_attribute("href"))

        return product_urls


In [6]:
for cat in all_category_link:
  print(cat)

https://www.truffleshuffle.co.uk/new-in
https://www.truffleshuffle.co.uk/best-sellers
https://www.truffleshuffle.co.uk/movies
https://www.truffleshuffle.co.uk/tv-shows
https://www.truffleshuffle.co.uk/cartoons
https://www.truffleshuffle.co.uk/music
https://www.truffleshuffle.co.uk/gaming
https://www.truffleshuffle.co.uk/fairytales-and-books
https://www.truffleshuffle.co.uk/iconic-brands
https://www.truffleshuffle.co.uk/games-toys
https://www.truffleshuffle.co.uk/superheroes-and-villains
https://www.truffleshuffle.co.uk/horror
https://www.truffleshuffle.co.uk/icons
https://www.truffleshuffle.co.uk/cartoons/anime
https://www.truffleshuffle.co.uk/food-and-drink-brands
https://www.truffleshuffle.co.uk/everything-else
https://www.truffleshuffle.co.uk/t-shirts-and-clothing
https://www.truffleshuffle.co.uk/homewares
https://www.truffleshuffle.co.uk/accessories
https://www.truffleshuffle.co.uk/gifts
https://www.truffleshuffle.co.uk/nightwear-underwear
https://www.truffleshuffle.co.uk/games-puz

In [7]:
final_all_links = []
for index, link in enumerate(all_category_link):
    print(f"scrap index: {index} on this link: {link}")
    product_urls = scrape_truffleshuffle(link)
    final_all_links += product_urls

# Print the list of product URLs


# Close the driver
# driver.close()


scrap index: 0 on this link: https://www.truffleshuffle.co.uk/new-in
scrap index: 1 on this link: https://www.truffleshuffle.co.uk/best-sellers
scrap index: 2 on this link: https://www.truffleshuffle.co.uk/movies
scrap index: 3 on this link: https://www.truffleshuffle.co.uk/tv-shows
scrap index: 4 on this link: https://www.truffleshuffle.co.uk/cartoons
scrap index: 5 on this link: https://www.truffleshuffle.co.uk/music
scrap index: 6 on this link: https://www.truffleshuffle.co.uk/gaming
scrap index: 7 on this link: https://www.truffleshuffle.co.uk/fairytales-and-books
scrap index: 8 on this link: https://www.truffleshuffle.co.uk/iconic-brands
scrap index: 9 on this link: https://www.truffleshuffle.co.uk/games-toys
scrap index: 10 on this link: https://www.truffleshuffle.co.uk/superheroes-and-villains
scrap index: 11 on this link: https://www.truffleshuffle.co.uk/horror
scrap index: 12 on this link: https://www.truffleshuffle.co.uk/icons
scrap index: 13 on this link: https://www.truffle

In [8]:
len(final_all_links)

10875

In [11]:
df = pd.DataFrame(final_all_links)
df.head(3)

,0
0,https://www.truffleshuffle.co.uk/product/36772...
1,https://www.truffleshuffle.co.uk/product/36816...
2,https://www.truffleshuffle.co.uk/product/36815...


In [12]:
df.to_csv('truffleshuffle_all.csv', index=False)

In [13]:
df = pd.read_csv('/content/truffleshuffle_all.csv')
df.head(3)

,0
0,https://www.truffleshuffle.co.uk/product/36772...
1,https://www.truffleshuffle.co.uk/product/36816...
2,https://www.truffleshuffle.co.uk/product/36815...


In [14]:
df.describe()

,0
count,10875
unique,1758
top,https://www.truffleshuffle.co.uk/product/36003...
freq,11


In [16]:
df = df.drop_duplicates(keep='first')
df.reset_index(drop=True, inplace=True)

In [42]:
df.describe()

,0
count,1758
unique,1758
top,https://www.truffleshuffle.co.uk/product/36772...
freq,1


In [43]:
# save and post data in db
for index, row in df.iterrows():
  print(index)
  get_link_details(row[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721


In [35]:
# url = "https://www.truffleshuffle.co.uk/product/34340/teenage-mutant-ninja-turtles-all-over-print-lounge-pants?refSrc=34360&nosto=productpage-nosto-2"
# final_images, title,  description = get_link_details(url)
# print(description)

Now, there are times when even Michelangelo, Donatello, Leonardo and Raphael aren't fighting Shredder and the other bad guys in Teenage Mutant Ninja Turtles, and they just want to kick back with some pizza. And we dare say you have those times as well - which is why you'll be obsessed with these exclusively designed in-house lounge pants with an epic all-over print of our four heroes in a half-shell. Get these super-soft pants onto enjoy your next night in, but just remember: wise men say “Forgiveness is divine, but never pay full price for late pizza!”
